# Meddling around with gendered names

**NOTES:** 

fasttext-wiki and w2v-googlenews has both upper and lower case names

glove-twitter, glove-wiki are lowercased everywhere

conceptnet has no names??

In [2]:
import gc
import gensim
import gensim.downloader as download_api
import json
import numpy as np
import pandas as pd
import sklearn
from sklearn import cluster
import time
import urllib.request

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 100)

df =  pd.read_csv("us-likelihood-of-gender-by-name-in-2014.csv")
df['nameLower'] = df.name.str.lower()
df.sex = df.sex.map({'F':0, 'M':1})
print("# names: ", len(df.name.unique()))

# names:  11316


Query the [name-prism](name-prism.com/) API to get probable ethnicity from name. [Ethnicolr](https://github.com/appeler/ethnicolr) has bad results and breaks your tensorflow installation -- don't use it.

In [13]:
def getEthnicity(name: str):
    """
    get ethinicity data for a name from name-prism.com API
    
    args: Name (str)
    returns: dict[str -> float]
             which represents [ethnicity -> probability]
    """
#     time.sleep(15)
    print(name)
    try:
        return dict(json.loads(
                urllib.request.urlopen(
                  "http://www.name-prism.com/api_eth/json/{0}".format(name)
                ).read()))
    # on failure, return empty dict
    except Exception as e:
        print("Problem: ", name)
        print(e)
        return {}

def maxValKey(d: dict):
    """ 
    Get the key with the maximal value from a dictionary
    """
    if not d: # if empty dict, return NaN (missing)
        return None
    vals=list(d.values())
    keys=list(d.keys())
    return keys[vals.index(max(vals))]

# create empty column to fill with ethnicity data
df['ethnicity'] = np.nan
df['ethnicity.prob'] = 1

In [14]:
for i in range(len(df.iloc[:3])):
    ethdict = getEthnicity(df['name'].iloc[i])
    print(ethdict)
    maxkey = maxValKey(ethdict)
    if maxkey:
        maxprob = ethdict[maxkey]
        df['ethnicity'].iloc[i] = maxkey
        df['ethnicity.prob'].iloc[i] = maxprob

Elaine
Problem:  Elaine
Expecting value: line 1 column 1 (char 0)
{}
Cathy
Problem:  Cathy
Expecting value: line 1 column 1 (char 0)
{}
Heidi
Problem:  Heidi
Expecting value: line 1 column 1 (char 0)
{}


In [ ]:
# gensim models from https://github.com/RaRe-Technologies/gensim-data
model_names = [
    ['glove-twitter-50', "nameLower"],
    ['glove-twitter-200', "nameLower"],
    ['glove-wiki-gigaword-50', "nameLower"],
    ['glove-wiki-gigaword-300', "nameLower"],
    ['word2vec-google-news-300', "nameLower"],
    ['word2vec-google-news-300', "name"],
    ['fasttext-wiki-news-subwords-300', "nameLower"],
    ['fasttext-wiki-news-subwords-300', "name"],
]
for m in range(len(model_names)):
    model_name = model_names[m][0]
    model = download_api.load(model_name)
    name_col = model_names[m][1]
    nameList = df[name_col].unique()
    # pre allocate memory for efficiency
    dn = pd.DataFrame(np.zeros(
        (len(nameList),
         model.vector_size))) # Add names to the dataframe to keep track
    dn.index = nameList
    oovNames = [] # out of model vocab names
    for name in nameList:
        try:
            dn.loc[dn.index==name, :] = model[name]
        except KeyError:
            oovNames.append(name)
    # clean result to cluster properly
    dn = dn.loc[~dn.index.isin(oovNames)]
    # get clustering
    genderClusters = sklearn.cluster.k_means(dn, 2)[1]
    # align result to join with M/F labellings
    dn = pd.DataFrame(list(zip(genderClusters, dn.index)), 
                      columns=[model_name +"_"+ name_col, name_col])
    # Merge results into final table
    # outer join has NaN for oov names
    df = df.merge(dn, on=name_col, how="outer")
    # cleanup: never trust python GC with big objects
    model, dn = None, None
    gc.collect()
    print('"{0}"'.format(model_name), " OOV words:", len(oovNames))

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
from sklearn import cluster, metrics

n_clusters = test_set.CAMPAIGNID.nunique()

test_set['kms'] = cluster.KMeans(n_clusters=n_clusters
                        ).fit(SE).labels_
test_set['acs'] = cluster.AgglomerativeClustering(n_clusters=n_clusters, 
                                                  affinity='euclidean', linkage='ward'
                        ).fit(SE).labels_
# NOTE: spectral requres hyperparameter fiddling to get good performance
test_set['spc'] = cluster.SpectralClustering(n_clusters=n_clusters,
                                             affinity= 'poly',# 'nearest_neighbors',
                                             n_init=50, 
                                             degree=3, gamma=0.3,
                                             eigen_solver='arpack', # optional (faster)
                        ).fit(SE).labels_


print("adj. MI score:   {0} | {1} | {2}".format(
    int(metrics.adjusted_mutual_info_score(test_set['kms'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_mutual_info_score(test_set['acs'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_mutual_info_score(test_set['spc'], test_set['CAMPAIGNID']) * 100),
))
print("adj. RAND score: {0} | {1} | {2}".format(
    int(metrics.adjusted_rand_score(test_set['kms'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_rand_score(test_set['acs'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_rand_score(test_set['spc'], test_set['CAMPAIGNID']) * 100),
))
print("F-M score:       {0} | {1} | {2}".format(
    int(metrics.fowlkes_mallows_score(test_set['kms'], test_set['CAMPAIGNID']) * 100),
    int(metrics.fowlkes_mallows_score(test_set['acs'], test_set['CAMPAIGNID']) * 100),
    int(metrics.fowlkes_mallows_score(test_set['spc'], test_set['CAMPAIGNID']) * 100),
))